In [37]:
from networkParser import parseContacts

# contactList, biteList, timeList = parseContacts("GMAP0/CatemacoBaseline_HET 36756.xml")
contactList, biteList, timeList = parseContacts("data/GMAP0.50/CatemacoBaseline_HET 68944.xml")
contactList = sorted(contactList)

## Set SEIR colors

In [38]:
colors = {
    "susceptible" : 0x00C853, #green
    "exposed" : 0xF57F17,     #orange
    "infected" : 0xD50000,        #infected
    "recovered" : 0x2962FF,     #recovered
}

In [39]:
import random

graph = {
    "nodes": {},
    "edges": [],
}

for person in contactList:
    graph["nodes"][person] = {'color': colors["susceptible"], 'size': 2.0}
    
randomInfected = random.sample(contactList,3)
for infected in randomInfected:
    graph["nodes"][infected]['color'] = colors["infected"]

In [40]:
import jgraph
jgraph.draw(graph)

In [41]:
# WARMUP = int(random.uniform(25000, 70000))
WARMUP = int(random.uniform(4000, 10000))
infectionsPerContact = {}

for i in contactList:
    infectionsPerContact[i] = []

timeListSortedKeys = list(filter(lambda x: x > WARMUP,  sorted(timeList.keys())))

exposed = {}

initialTime = 0 if len(timeListSortedKeys) == 0 else int(timeListSortedKeys[0])

# print(timeList)
# print(timeListSortedKeys)

In [42]:
infectedPeople = {}
susceptible = contactList.copy()

for i in randomInfected:
        infectedPeople[i] = initialTime
        susceptible.remove(i)
recovered = {}
asymptomatic = {}
infectedMosquitoes = []
mosquitoesContact = {}
SEIR = {}
edges = []

In [43]:
fourDays = 1152
threeDays = 864
tenDays = 2880

In [44]:
for key in timeListSortedKeys:

		currentMosquitoes = timeList[key]
		currTime = int(key)
		for mosquito in currentMosquitoes:
			# Get time/bite
			currentBiteList = biteList[mosquito]
			i = currentBiteList["times"].index(key)
			bited = currentBiteList["bites"][i]
			
			if(bited in infectedPeople and mosquito not in infectedMosquitoes):
				# pVH Transmission probability from an infectious human to a susceptible mosquito per bite    0.3–0.75   
				 # (Gao et al., 2016; Andraud et al., 2012; Chikaki and Ishikawa, 2009)
				probability = random.uniform(0.3, 0.75)
				if(random.uniform(0.0,1.0) <= probability):
					infectedMosquitoes.append(mosquito)
					mosquitoesContact[mosquito] = bited
					# print("Mosquito {} got infected by {} at time {}".format(mosquito, bited, currTime))
			else:
				# pHV Transmission probability from an infectious mosquito to a susceptible human per bite    0.1–0.75    
				# (Gao et al., 2016; Andraud et al., 2012; Chikaki and Ishikawa, 2009)

				# 1/fH    Duration of human latent period, E (days)   4   (Turmel et al., 2016; Bearcroft, 1956)
				# 345600
				if(mosquito in infectedMosquitoes and bited in susceptible):
					if(random.uniform(0.0,1.0) <= random.uniform(0.1,0.75)):
						exposed[bited] = currTime
						susceptible.remove(bited)
						edges.append((mosquitoesContact[mosquito], currentBiteList["bites"][i]))
						infectionsPerContact[mosquitoesContact[mosquito]].append(bited)
						# print("Human {} got infected by {} {} at time {}".format(currentBiteList["bites"][i], mosquito, mosquitoesContact[mosquito], currTime))
						# status()
						# draw_graph(contactList, edges, infectedPeople, susceptible, exposed, recovered)
		removals = []
		for e in exposed:
			# https://www.sciencedirect.com/science/article/pii/S1755436517301093
			if(currTime >= exposed[e]+ (threeDays * 2) ):
			# if(currTime >= exposed[e]+ (threeDays * 1) ):
				infectedPeople[e] = currTime
				removals.append(e)
				# print("{} went from exposed to infected at {}".format(e,exposed[e]+fourDays))
				
		for r in removals:
			exposed.pop(r)
			# draw_graph(contactList, edges, infectedPeople, susceptible, exposed, recovered)


		removals = []
		for e in infectedPeople:
			if(currTime >= infectedPeople[e]+ (fourDays + threeDays)):
			# if(currTime >= infectedPeople[e]+ (fourDays)):
				recovered[e] = currTime
				removals.append(e)
				# print("{} went from infectious to recovered at time {}".format(e, infectedPeople[e]+fourDays))
		for r in removals:
			infectedPeople.pop(r)
			# draw_graph(contactList, edges, infectedPeople, susceptible, exposed, recovered)

		# UPDATE SEIR
		SEIR[currTime] = [len(susceptible), len(exposed), len(infectedPeople), len(recovered)]

	# Fin de simulación de epidemia

In [45]:
print(infectionsPerContact)

{'aa': [], 'ab': [], 'ac': [], 'ad': [], 'ae': [], 'af': [], 'ag': [], 'ah': [], 'ai': [], 'aj': [], 'ak': [], 'al': [], 'am': [], 'an': [], 'ao': [], 'ap': [], 'aq': [], 'ar': [], 'as': [], 'at': [], 'au': [], 'av': [], 'aw': [], 'ax': [], 'ay': [], 'ba': [], 'bb': [], 'bc': [], 'bd': [], 'be': [], 'bf': [], 'bg': [], 'bh': [], 'bi': [], 'bj': [], 'bk': [], 'bl': [], 'bm': [], 'bn': ['ah', 'cb'], 'bo': [], 'bp': [], 'bq': [], 'br': [], 'bs': [], 'bt': [], 'bu': [], 'bv': ['bw', 'cg'], 'bw': [], 'bx': [], 'by': [], 'bz': [], 'ca': [], 'cb': [], 'cc': ['cj'], 'cd': [], 'ce': [], 'cf': [], 'cg': [], 'ch': [], 'ci': [], 'cj': ['ag'], 'ck': ['bn', 'bc', 'bv', 'ai', 'by']}


In [46]:
print("Exposed:", exposed)
print("Infected: ", infectedPeople)
print("Recovered: ", recovered)


Exposed: {'cb': 18709}
Infected:  {'cg': 19694}
Recovered:  {'ck': 7670, 'aw': 7670, 'cc': 7670, 'bn': 12558, 'bc': 12905, 'bv': 13030, 'cj': 13503, 'ai': 14645, 'by': 17189, 'ag': 18059, 'bw': 18116, 'ah': 18655}


<font size="18"><br>
    Graph Showing:<br><br></font>
    <font size="18" color="green">Susceptible<br><br></font>
    <font size="18" color="orange">Exposed<br><br></font> 
    <font size="18" color="red">Infected<br><br></font>
    <font size="18" color="blue">Recovered<br><br></font>

In [49]:
for infectedKey in infectedPeople.keys():
    graph["nodes"][infectedKey]["color"] = colors["infected"]
    
for recoveredKey in recovered.keys():
    graph["nodes"][recoveredKey]["color"] = colors["recovered"]

for exposedKey in exposed.keys():
    graph["nodes"][exposedKey]["color"] = colors["exposed"]
    
for infectionKey in infectionsPerContact.keys():
    infection = infectionsPerContact[infectionKey]
    for outNode in infection:
        graph["edges"].append({"source": infectionKey, "target": outNode, "color": 0x000000})
jgraph.draw(graph)